## Update a Hosted Table

## AOIs: Update conservation concern on species lookup tables.
We have a lookup table per taxa (reptiles, amphibians, birds and mammlas) with a column **conservation concern** of each species, and we have a separate lookup table with a column **has_image**. We need both columns to be on the same table

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

In [3]:
# env_path = "../../.env"
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [4]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [5]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


In [5]:
## Tables with conservation concern (cc)
cc_keys = {'amphibians':'eb487fb505e34052b4cb9e02f3f7a22c',
       'reptiles':'38356d976d3f43d7a0d2ab91034b054',
       'mammals':'f6e7514c775442b39274d306b54a5952',
       'birds':'71e61cd2211b4670a28bfb14b3693f66'}
## Tables with has_image (hi)
hi_keys = {'amphibians':'a641a4cd269345dea93b8bcb1cb66676',
       'reptiles':'81c72a2a5ee6413699960b4c4bd9540f',
       'mammals':'84d3c71caf97479d85f620a4ee217d68',
       'birds':'4d8698734b654bb9bb7a61d9af314c76'}

In [6]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [59]:
## amphibians
cc = getHTfromId(cc_keys['amphibians'])
hi = getHTfromId(hi_keys['amphibians'])

In [60]:
cc.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,ObjectId
0,Acanthixalus_sonjae,1,Acanthixalus sonjae,787,787,100,100,0,0,1
1,Acanthixalus_spinosus,2,Acanthixalus spinosus,1041435,207623,20,15,0,0,2
2,Acris_crepitans,3,Acris crepitans,1750647,90682,5,15,0,10,3
3,Acris_gryllus,4,Acris gryllus,494322,24443,5,15,0,10,4
4,Adelastes_hylonomos,5,Adelastes hylonomos,155,155,100,100,0,0,5


In [61]:
hi.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,ObjectId
0,Acanthixalus_sonjae,1,Acanthixalus sonjae,787,787,100,100,0,0,1
1,Acanthixalus_spinosus,2,Acanthixalus spinosus,1041435,207623,20,15,0,0,2
2,Acris_crepitans,3,Acris crepitans,1750647,90682,5,15,0,1,3
3,Acris_gryllus,4,Acris gryllus,494322,24443,5,15,0,1,4
4,Adelastes_hylonomos,5,Adelastes hylonomos,155,155,100,100,0,0,5


In [66]:
### Add conservation concern to hi table
hi = hi.merge(cc[['SliceNumber','conservation_concern']],how='inner',on='SliceNumber')

In [91]:
### Add new field to Hosted service
## Create fields
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = sdf = flayer.query().sdf
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            current_field['length'] = 45000
        if dt == 'int64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [119]:
item = gis.content.get(hi_keys['amphibians'])
flayer = item.tables[0]
fields_to_be_added = createFieldsToBeAdded(flayer, hi)

In [120]:
fields_to_be_added

[{'name': 'conservation_concern',
  'type': 'esriFieldTypeDouble',
  'actualType': 'nvarchar',
  'alias': 'conservation_concern',
  'sqlType': 'sqlTypeOther',
  'length': 4000,
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None}]

In [121]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [158]:
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [ ]:
features_for_update = createFeaturesForUpdate(flayer = flayer ,
                        csv_table = hi,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")

In [ ]:
flayer.edit_features(updates= features_for_update)

In [160]:
## Tables with conservation concern (cc)
cc_keys = {'birds':'71e61cd2211b4670a28bfb14b3693f66'}
## Tables with has_image (hi)
hi_keys = {'birds':'4d8698734b654bb9bb7a61d9af314c76'}

In [182]:
### reptiles
hi_keys = {'reptiles':'81c72a2a5ee6413699960b4c4bd9540f'}
cc_keys = {'reptiles':'38356d976d3f43d7a0d2ab91034b054b'}

In [183]:
for key in cc_keys:
    ## get df from Hosted tables
    cc = getHTfromId(cc_keys[key])
    hi = getHTfromId(hi_keys[key])
    
    ### bring conservation_concern column to base table
    hi = hi.merge(cc[['SliceNumber','conservation_concern']],how='inner',on='SliceNumber')
    
    ### Create New fields
    item = gis.content.get(hi_keys[key])
    flayer = item.tables[0]
    fields_to_be_added = createFieldsToBeAdded(flayer, hi)
    
    ### Add new fields
    flayer.manager.add_to_definition({'fields':fields_to_be_added})
    
    ### Create features to update
    #### triplicate columns, check what has happened?
    if key == 'birds':
        start = np.arange(0,len(hi), step = 2000)
        for i in start:
            features_for_update = createFeaturesForUpdate(flayer = flayer, 
                                                          csv_table = hi[i:i+2000], 
                                                          fields_to_be_added =  fields_to_be_added, 
                                                          id_field_in_csv = "SliceNumber", 
                                                          id_field_in_service = "SliceNumber")
            flayer.edit_features(updates= features_for_update)
    else: 
        features_for_update = createFeaturesForUpdate(flayer = flayer,
                        csv_table = hi,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")
    
        ### Update Features
        flayer.edit_features(updates= features_for_update)
    

## Update common name and synonym column from downloaded CSV

In [9]:
### Example for amphibians table

table = pd.read_csv('/Users/sofia/Documents/HE_Data/Amphibian_CRF_species_table.csv') # table with new fields
### coming from table on AGOL: https://eowilson.maps.arcgis.com/home/item.html?id=b09984e429814a0ea7ea5ed44dd3b609

In [14]:
table.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,common_name,synonyms
0,Acanthixalus_sonjae,1,Acanthixalus sonjae,787,787,100,100,0,0,NaN,NaN
1,Acanthixalus_spinosus,2,Acanthixalus spinosus,1041435,207623,20,15,0,0,NaN,NaN
2,Acris_crepitans,3,Acris crepitans,1750647,90682,5,15,0,1,Northern Cricket Frog,NaN
3,Acris_gryllus,4,Acris gryllus,494322,24443,5,15,0,1,"Southern Cricket Frog, Florida Cricket Frog (...",NaN
4,Adelastes_hylonomos,5,Adelastes hylonomos,155,155,100,100,0,0,NaN,NaN


In [12]:
## Function that removes parenthesis (and anything inside them), and leading and trailing spaces
import re
def make_array(row):
    row_list = row.split(',')
    row_array= []
    for i in row_list: 
        row_array.append(re.sub(r'\([^()]*\)', '', i).rstrip().lstrip())
    return row_array

In [20]:
## Apply function to all rows in table and create an array of names
table['common_name_array']= np.nan
table['common_name_array'] = table['common_name'].apply(lambda row : json.dumps(make_array(row)) if type(row)==str else row)
table['synonyms_array']= np.nan
table['synonyms_array']= table['synonyms'].apply(lambda row : json.dumps(make_array(row)) if type(row)==str else row)

In [17]:
table.common_name[3]

'Southern Cricket Frog, Florida  Cricket Frog (<i>A. g. dorsalis</i>), Southern Cricket Frog (<i>A. g. gryllus</i>)'

In [26]:
table.common_name_array[3]

'["Southern Cricket Frog", "Florida  Cricket Frog", "Southern Cricket Frog"]'

In [21]:
table.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,common_name,synonyms,common_name_array,synonyms_array
0,Acanthixalus_sonjae,1,Acanthixalus sonjae,787,787,100,100,0,0,NaN,NaN,NaN,NaN
1,Acanthixalus_spinosus,2,Acanthixalus spinosus,1041435,207623,20,15,0,0,NaN,NaN,NaN,NaN
2,Acris_crepitans,3,Acris crepitans,1750647,90682,5,15,0,1,Northern Cricket Frog,NaN,"[""Northern Cricket Frog""]",NaN
3,Acris_gryllus,4,Acris gryllus,494322,24443,5,15,0,1,"Southern Cricket Frog, Florida Cricket Frog (...",NaN,"[""Southern Cricket Frog"", ""Florida Cricket Fr...",NaN
4,Adelastes_hylonomos,5,Adelastes hylonomos,155,155,100,100,0,0,NaN,NaN,NaN,NaN


In [ ]:
## Function to get table from AGOL as sdf
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [ ]:
## Get sdf of amphibians table
amph_key = 'a641a4cd269345dea93b8bcb1cb66676'
amph = getHTfromId(amph_key) # get table amphibians from AGOL (table to which we want add new field)


In [28]:
amph.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,ObjectId
0,Anodonthyla_hutchisoni,351,Anodonthyla hutchisoni,374,343,92,100,0,0,1
1,Amolops_wuyiensis,301,Amolops wuyiensis,75331,0,0,33,0,0,2
2,Ameerega_picta,201,Ameerega picta,1374742,609286,44,15,0,1,3
3,Allobates_fuscellus,101,Allobates fuscellus,564464,304538,54,15,0,0,4
4,Alytes_cisternasii,151,Alytes cisternasii,149939,48900,33,23,0,1,5


In [27]:
### Add common name and synonym columns to amphibians table
amph2 = amph.merge(table[['SliceNumber','common_name_array', 'synonyms_array']],how='inner',on='SliceNumber')
amph2.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,ObjectId,common_name_array,synonyms_array
0,Anodonthyla_hutchisoni,351,Anodonthyla hutchisoni,374,343,92,100,0,0,1,NaN,NaN
1,Amolops_wuyiensis,301,Amolops wuyiensis,75331,0,0,33,0,0,2,NaN,NaN
2,Ameerega_picta,201,Ameerega picta,1374742,609286,44,15,0,1,3,NaN,NaN
3,Allobates_fuscellus,101,Allobates fuscellus,564464,304538,54,15,0,0,4,NaN,NaN
4,Alytes_cisternasii,151,Alytes cisternasii,149939,48900,33,23,0,1,5,"[""Iberian midwife toad"", ""Ibersiche geburtshel...",NaN


In [29]:
### Add new field to Hosted service
## Create fields
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = sdf = flayer.query().sdf
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            current_field['length'] = 45000
        if dt == 'int64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [30]:
item = gis.content.get(amph_key)
flayer = item.tables[0]
fields_to_be_added = createFieldsToBeAdded(flayer, amph2)

In [31]:
fields_to_be_added

[{'name': 'common_name_array',
  'type': 'esriFieldTypeString',
  'actualType': 'nvarchar',
  'alias': 'common_name_array',
  'sqlType': 'sqlTypeOther',
  'length': 45000,
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None},
 {'name': 'synonyms_array',
  'type': 'esriFieldTypeString',
  'actualType': 'nvarchar',
  'alias': 'synonyms_array',
  'sqlType': 'sqlTypeOther',
  'length': 45000,
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None}]

In [32]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [33]:
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [34]:
features_for_update = createFeaturesForUpdate(flayer = flayer ,
                        csv_table = amph2,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")

351 not available in service
301 not available in service
201 not available in service
101 not available in service
151 not available in service
1 not available in service
302 not available in service
352 not available in service
51 not available in service
251 not available in service
102 not available in service
202 not available in service
152 not available in service
2 not available in service
303 not available in service
353 not available in service
52 not available in service
252 not available in service
103 not available in service
203 not available in service
153 not available in service
3 not available in service
304 not available in service
354 not available in service
53 not available in service
253 not available in service
104 not available in service
204 not available in service
154 not available in service
4 not available in service
355 not available in service
54 not available in service
254 not available in service
105 not available in service
205 not available in servi

In [35]:
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 31,
   'uniqueId': 31,
   'globalId': None,
   'success': True},
  {'objectId': 31, 'uniqueId': 31, 'globalId': None, 'success': True},
  {'objectId': 264, 'uniqueId': 264, 'globalId': None, 'success': True},
  {'objectId': 264, 'uniqueId': 264, 'globalId': None, 'success': True},
  {'objectId': 832, 'uniqueId': 832, 'globalId': None, 'success': True},
  {'objectId': 832, 'uniqueId': 832, 'globalId': None, 'success': True},
  {'objectId': 874, 'uniqueId': 874, 'globalId': None, 'success': True},
  {'objectId': 874, 'uniqueId': 874, 'globalId': None, 'success': True},
  {'objectId': 1905, 'uniqueId': 1905, 'globalId': None, 'success': True},
  {'objectId': 1905, 'uniqueId': 1905, 'globalId': None, 'success': True},
  {'objectId': 2324, 'uniqueId': 2324, 'globalId': None, 'success': True},
  {'objectId': 2324, 'uniqueId': 2324, 'globalId': None, 'success': True},
  {'objectId': 2472, 'uniqueId': 2472, 'globalId': None, 'success': True},
